In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv("train.csv", index_col=0)
card = pd.read_csv("card.csv")
test = pd.read_csv("test.csv", index_col=0)
user = pd.read_csv("user.csv")

In [ ]:
train

In [ ]:
card

In [ ]:
test

In [ ]:
user

In [ ]:
# 'user_id' と 'card_id' を文字列に変換して連結
train['unique_id'] = train['user_id'].astype(str) + '_' + train['card_id'].astype(str)
test['unique_id'] = test['user_id'].astype(str) + '_' + test['card_id'].astype(str)
card['unique_id'] = card['user_id'].astype(str) + '_' + card['card_id'].astype(str)

# 混同しないように一旦列を削除
train = train.drop(['user_id', 'card_id'], axis=1)
test = test.drop(['user_id', 'card_id'], axis=1)
card = card.drop(['user_id', 'card_id'], axis=1)

# 分割できるようにフラグ列を作成
train['is_train'] = 1
test['is_train'] = 0

In [ ]:
# trainとtestを結合
train_test = pd.concat([train, test])

# cardデータフレームとマージ
merged_with_card = pd.merge(train_test, card, on='unique_id', how='left')

In [ ]:
merged_with_card

In [ ]:
# 'unique_id'から'user_id'を抽出
merged_with_card['user_id'] = merged_with_card['unique_id'].apply(lambda x: int(x.split('_')[0]))

# userデータフレームとマージ
final_merged = pd.merge(merged_with_card, user, on='user_id', how='left')

In [ ]:
final_merged

In [ ]:
final_merged.info()

In [ ]:
final_merged['year_pin_last_changed']

In [ ]:
# merchant_cityとmerchant_stateに基づいて最も頻繁に出現するzipコードを取得
merchant_zip_map = final_merged.dropna(subset=['zip']).groupby(['merchant_city', 'merchant_state'])['zip'].agg(lambda x: x.mode()[0] if not x.mode().empty else None).to_dict()

# cityとstateに基づいて最も頻繁に出現するzipcodeを取得
user_zip_map = final_merged.dropna(subset=['zipcode']).groupby(['city', 'state'])['zipcode'].agg(lambda x: x.mode()[0] if not x.mode().empty else None).to_dict()

# 欠損値を埋める関数
def fill_missing_zip(row):
    if pd.isna(row['zip']):
        # "ONLINE"という特殊なケースを処理
        if row['merchant_city'] == 'ONLINE' and pd.isna(row['merchant_state']):
            return row['zip']
        merchant_zip = merchant_zip_map.get((row['merchant_city'], row['merchant_state']))
        user_zip = user_zip_map.get((row['city'], row['state']))
        return merchant_zip if merchant_zip is not None else (user_zip if user_zip is not None else None)
    return row['zip']

# apply関数を用いてzip列の欠損値を埋める
final_merged['zip'] = final_merged.apply(fill_missing_zip, axis=1)


In [ ]:
final_merged.info()

In [ ]:
import time

# キャッシュ用の辞書
lat_lng_cache = {}

def get_lat_lng(zip_code):
    # キャッシュ内に結果があるか確認
    if zip_code in lat_lng_cache:
        return lat_lng_cache[zip_code]

    try:
        geocode_result = gmaps.geocode('{}, USA'.format(zip_code))
        lat = geocode_result[0]["geometry"]["location"]["lat"]
        lng = geocode_result[0]["geometry"]["location"]["lng"]
        lat_lng_cache[zip_code] = (lat, lng)
        return lat, lng
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

# 進捗確認用
total_rows = len(final_merged)
progress_step = total_rows // 10

# DataFrameに新しい列として緯度と経度を追加
for i, row in enumerate(final_merged.itertuples()):
    if i % progress_step == 0:
        print(f"Progress: {i}/{total_rows}")

    zip_code = row.zip
    final_merged.at[row.Index, 'shop_lat'], final_merged.at[row.Index, 'shop_lng'] = get_lat_lng(zip_code)
    time.sleep(0.1)  # APIのレート制限に対応するための遅延
